This is an attempt to recreate the neural network predicting concrete strength development [1] based on the experimental dataset [2].

Unlike in [1], I didn't divide the whole dataset into 4 subsets based on local correlation. Instead, I pick a random 3/4 of the experiments from the dataset and use those for validation of the trained network. All other experiments are used for training the network. It turns out that this simple randomised approach works with an additional hidden layer of a few neurons that model an experiment classifier whose purpose is to learn a few clusters within the training dataset. The error rate converges to a fixed limit. The predicted values are fairly correlated with the experimental results. The coefficients of determination R^2 are about 0.85 on the training set and about 0.8 on the validation set, which matches the accuracy of the NN in [1].

I use Python mostly for presentation reasons and because anybody can train this NN straight from GitHub with no local setup.

[1] I-Cheng Yeh, [Modeling of strength of high performance concrete using artificial neural networks](https://www.researchgate.net/publication/222447231_Modeling_of_Strength_of_High-Performance_Concrete_Using_Artificial_Neural_Networks_Cement_and_Concrete_research_2812_1797-1808), _Cement and Concrete Research_, Vol. 28, No. 12, pp. 1797-1808 (1998).

[2] https://archive.ics.uci.edu/ml/datasets/Concrete+Compressive+Strength

First we import the required Python libraries.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas
import seaborn
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, backend
from tensorflow.keras.layers.experimental import preprocessing

Make `numpy` values easier to read.

In [ ]:
np.set_printoptions(precision=3, suppress=True)

Load the dataset from [2]. I converted it to CSV for simplicity.

In [ ]:
dataset = pandas.read_csv("https://raw.githubusercontent.com/vkomenda/concrete-strength/main/concrete-dataset.csv")

Print some dataset statistics.

In [ ]:
dataset.describe().transpose()

Plot the distribution of the amounts of cement and water in the mix and their relative ratios.

In [ ]:
seaborn.pairplot(dataset[['Cement (kg/m^3)', 'Water (kg/m^3)']], diag_kind='kde')

The first few entries after shuffling, and their description.

In [ ]:
dataset.head()

Perform a random permutation on the dataset. Different runs of this program lead to different weights computed for the network because due to the random permutation the function that the NN tries to approximate is different in each case.

Split the features and labels into two parts: a 3/4 of the records for validation and 1/4 the rest for training.

The input parameters are those in the first 8 data columns (not counting the experiment indices on the left). The experimental results are in the rightmost column. These two categories are called features and labels respectively.

In [ ]:
train_set = dataset.sample(frac = 0.75)
valid_set = dataset.drop(train_set.index)

label_name = 'Concrete compressive strength (MPa)'
features_train = train_set.copy()
labels_train = features_train.pop(label_name)
features_valid = valid_set.copy()
labels_valid = features_valid.pop(label_name)

print(features_train.shape)
print(features_valid.shape)

Define the [coefficient of determination](https://en.wikipedia.org/wiki/Coefficient_of_determination) R^2 which is a measure of how well the observed outcomes are replicated by the model. It was used in [1] to show that the NN was more accurate than the regression analysis model.

In [ ]:
def R_squared(y, y_pred):
    residual = tf.math.reduce_sum(np.square(np.subtract(y, y_pred)))
    total = tf.math.reduce_sum(np.square(np.subtract(y, tf.math.reduce_mean(y))))
    r2 = np.subtract(1.0, np.divide(residual, total))
    return r2

def coeff_determination(y_true, y_pred):
    SS_res =  backend.sum(backend.square(y_true - y_pred)) 
    SS_tot = backend.sum(backend.square(y_true - backend.mean(y_true))) 
    return (1 - SS_res / (SS_tot + backend.epsilon()))

The loss function can be just the distance from the ideal R^2.

In [ ]:
def my_loss(y_true, y_pred):
    r2 = coeff_determination(y_true, y_pred)
    return 1 - r2

Define and train the NN model. There are 8 input neurons. Those are connected to 32 neurons in the hidden (intermediate) layer. Those are in turn connected to a single output neuron.

All connections are labelled with weights which are the values that change during training to output the closest result according to the loss function - mean squared error in this case.

In [ ]:
model = tf.keras.Sequential([
    layers.Dense(32, activation = lambda x: tf.keras.activations.relu(x, alpha=0.1)),
    layers.Dense(16, activation = lambda x: tf.keras.activations.relu(x, alpha=0.1)),
    layers.Dense(1)
])

model.compile(
#    loss = keras.losses.MeanSquaredError(),
    loss = my_loss,
    metrics=[coeff_determination],
    optimizer = tf.optimizers.Adam(learning_rate = 0.001)
)

history = model.fit(
    features_train,
    labels_train,
    epochs=400,
    verbose=1,
    validation_data = (
        features_valid,
        labels_valid
    )
)

Print some model summary.

In [ ]:
model.summary()

Print the trained weights labelling the connections of the hidden layer neurons with the output neuron.

In [ ]:
model.layers[1].kernel.numpy().transpose()[0]

Here is another way to access the training history.

In [ ]:
hist = pandas.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

Define a plotter function for the training loss.

In [ ]:
def plot_loss(history):
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.ylim([0, 500])
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

Now plot the losses: the training set loss and the loss on the validation set. If the loss curve starts to "dance" after a few epochs of relative convergence, that means the NN is overfitted on the training data. That is, it becomes too specialised on possible outliers in the training set. That means that outliers in the validation set can get misclassified as a consequence.

In [ ]:
plot_loss(history)

Predict the concrete strength of the set of experiments used for validation. Those experiments weren't used for training, so the predictions shows the performance of the NN on real inputs.

In [ ]:
labels_predict = model.predict(features_valid)

Define a plotter function for the difference between what was observed in the experiments and what the NN actually computed.

In [ ]:
def plot_comparison():
    plt.scatter(labels_valid, labels_predict)
#    plt.plot(labels[:,0], y_pred_test, 'r', label='Predicted Data')
    plt.xlabel('strength observed in lab')
    plt.ylabel('strength predicted by NN')

Now plot this comparison.

In [ ]:
plot_comparison()

Print the validation concrete strengths alongside the predicted ones.

In [ ]:
y_valid = labels_valid.values
y_predict = labels_predict.transpose()[0]

print('validation values:\n', y_valid)
print('predicted values:\n', y_predict)

Compute the R^2 of the validation set. In [1] it was greater than 0.8. Here it is lower because we didn't split the dataset into subsets based on internal correlation.

In [ ]:
r2_valid = R_squared(y_valid, y_predict)
print('R^2 of the validation set:', r2_valid)